<a href="https://colab.research.google.com/github/RohithJ11/NLP_Privacy_Policies/blob/main/BARTupdated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets


In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

# Step 1: Load your dataset
dataset = load_dataset('RohithJ/cleaned_dataset_prvy_plcy')  # Replace with your actual dataset name

# Step 2: Preprocess the data
model_name = 'facebook/bart-large-cnn'  # BART model pre-trained for summarization
tokenizer = BartTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    # Tokenize the inputs and labels
    inputs = tokenizer(examples['Content'], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
    outputs = tokenizer(examples['Summary_of_Content'], max_length=128, truncation=True, padding="max_length", return_tensors="pt")

    # inputs and outputs now include padding and are truncated
    return {"input_ids": inputs['input_ids'], "attention_mask": inputs['attention_mask'], "labels": outputs['input_ids']}


tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Step 3: Define the model
model = BartForConditionalGeneration.from_pretrained(model_name)

# Step 4: Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    evaluation_strategy="epoch",     # Evaluation each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    weight_decay=0.01,               # Strength of weight decay
    save_total_limit=3,              # Only last 3 models are saved
    num_train_epochs=3              # Number of training epochs
)

# Initialize the Trainer with generation settings for evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Step 6: Train the model
trainer.train()



Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,2.329527
2,No log,2.027047
3,No log,1.888458


TrainOutput(global_step=39, training_loss=1.844606448442508, metrics={'train_runtime': 75.3347, 'train_samples_per_second': 2.031, 'train_steps_per_second': 0.518, 'total_flos': 331567004123136.0, 'train_loss': 1.844606448442508, 'epoch': 3.0})

In [10]:
model.save_pretrained('/content/drive/MyDrive/trained_model')
tokenizer.save_pretrained('/content/drive/MyDrive/trained_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/trained_model/vocab.json',
 '/content/drive/MyDrive/trained_model/merges.txt',
 '/content/drive/MyDrive/trained_model/added_tokens.json')

In [11]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the model and tokenizer
model_path = '/content/drive/MyDrive/trained_model'  # Adjust the path if necessary
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)


In [19]:
def summarize(text):
    # Encode the text into tensor of token ids
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate summary with the model
    summary_ids = model.generate(inputs, max_length=3000, min_length=200, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary


In [14]:
pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.7 MB/s eta 0:00:00


In [15]:
import pdfplumber
import re

def clean_text(text):
    # Replace double spaces with single spaces
    text = re.sub(r'\s+', ' ', text)
    # Replace double quotes with single quotes
    text = re.sub(r'\"+', '"', text)
    return text

def extract_text_from_pdf(pdf_path):
    all_text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:  # Ensure there's text on the page
                all_text += text + ' '  # Add a space between texts from different pages

    cleaned_text = clean_text(all_text)
    return cleaned_text

# Example usage
pdf_path = '/content/drive/MyDrive/Privacy_Policy_Pfizer.pdf'
only_text = extract_text_from_pdf(pdf_path)
print(only_text)


Pfizer Privacy Policy U.S. Effective August 23rd, 2023 This Privacy Policy describes the information that we collect about you on the website from which you linked (“Site”); how we use that information; how we protect it; and the choices you may make with respect to it. Except as specifically provided below in the Additional State Disclosures Section, this Privacy Policy does not apply to any other information collected by Pfizer by or through any other means, such as information collected offline. When we refer to ourselves as “we" or "Pfizer," we mean Pfizer Inc. and all of its subsidiary companies. Our Site may contain links to third party websites/content/services that are not owned or controlled by Pfizer. Pfizer is not responsible for how these properties operate or treat your personal data so we recommend that you read the privacy policies and terms associated with these third party properties carefully. California Residents: This Privacy Policy describes how we collect, use, di

In [20]:
# Example text
text = only_text

# Generate the summary
summary = summarize(text)
print("Summary:", summary)
print(only_text)


Summary: The Pfizer Privacy Policy U.S. is effective August 23, 2023. It outlines how Pfizer collects and uses personal data collected through its website, how it uses that information, and the choices made with respect to it. This Privacy Policy does not apply to information collected by Pfizer by or through other means, such as offline information collected through third-party websites or services. Data collected directly from users through registration, applications, surveys, and purchases, and from third parties such as healthcare providers, cookies, pixel tags, clear GIFs, or similar technologies, as well as browser type, operating system, browser settings, web pages viewed, interactions, interactions with content, interaction with webpages, links clicked, and other identifiers. Additionally, cookies and other tools are used to collect information about users' online activities over time and across different websites and services. Pfizer may augment personal data gathered from oth